In [6]:
import time, sys , os
import configparser

import pandas as pd

from dark import DarkGateway

sys.path.append("../")

from utils.oasis_reader import extract_itens_from_json
from clients import CrossrefClient, HyperDriveClient

# get input

In [ ]:
input_path="D:\\tmp\oaisis\\"
file_list = ['oasisbr_1.zip','oasisbr_2.zip','oasisbr_3.zip','oasisbr_4.zip','oasisbr_5.zip']
# file_list = ['oasisbr_5.zip']

target_inst = {
               'UEPB' : 1,
               'UFCG' : 1,
               'UFPB' : 1,
               'FACISA' : 1,
               'FIP' : 1
            # 'UFSC' : 1
}

output=[]

for i in file_list:
    input_file_path = input_path + i
    # out_local = extract_itens_from_json(input_file_path,remove_att_list=['description'])
    out_local = extract_itens_from_json(input_file_path, filtro_inst=target_inst,remove_att_list=['description'])
    output = output + out_local

explorar

- id - o id do documento no Oasisbr
- oai_identifier_str - id externo que vem do repositório de origem
- instname_str - nome da instituição de origem
- instacron_str - acrônimo da instituição de origem
- network_acronym_str - acrônimo da rede de origem (pode descartar)
- network_name_str - nome da rede origem (pode descartar)
- title - título
- author - autores
- publishDate  - data da publicação
- format - tipo de documento (article, masterThesis, doctoralThesis, …)
- description - abstract
- topic - keywords
- language - idioma do documento
- eu_rights_str_mv - tipo de acesso (openAccess, closed, …)
- url - url externa para o documento

In [ ]:
df_ = pd.DataFrame(output)
len(df_)

In [ ]:
df_.to_json('pb-data.json')
# df_.to_json('UFSC-data.json')
df_.head(2)
# df.instacron_str.value_counts()

# load

In [7]:
df=pd.read_json('pb-data.json')

In [ ]:
df

- id - o id do documento no Oasisbr
- oai_identifier_str - id externo que vem do repositório de origem
- instname_str - nome da instituição de origem
- instacron_str - acrônimo da instituição de origem
- network_acronym_str - acrônimo da rede de origem (pode descartar)
- network_name_str - nome da rede origem (pode descartar)
- title - título
- author - autores
- publishDate  - data da publicação
- format - tipo de documento (article, masterThesis, doctoralThesis, …)
- description - abstract
- topic - keywords
- language - idioma do documento
- eu_rights_str_mv - tipo de acesso (openAccess, closed, …)
- url - url externa para o documento

In [8]:
def getDarkGateway():
    PROJECT_ROOT = "./"
    bc_config = configparser.ConfigParser()
    deployed_contracts_config = configparser.ConfigParser()
    
    bc_config.read(os.path.join(PROJECT_ROOT, "config.ini"))
    deployed_contracts_config.read(os.path.join(PROJECT_ROOT, "deployed_contracts.ini"))
    dark_gw = DarkGateway(bc_config, deployed_contracts_config)
    return dark_gw

def measure_response_time(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    response_time = end_time - start_time
    return result, response_time

def measure_tx_params(dark_gw, func, *args, **kwargs):
    # initial_amount = dark_gw.get_account_balance()
    result, response_time = measure_response_time(func, *args, **kwargs)
    # final_amount = dark_gw.get_account_balance()

    # tx_value = initial_amount - final_amount
    tx_value = 0
    return result, response_time , tx_value

# def adicionar(a, b):
#     return a + b

# Exemplo de uso:
# resultado, tempo_de_resposta = measure_response_time(adicionar, 5, 3)

In [10]:
i=0
cc = HyperDriveClient()
oasis_dark_map = {}

 
dark_gw = getDarkGateway()

for index, row in df.iterrows():
    oasis_id = row['id']
    oai_id   = row['oai_identifier_str']
    title    = row.get('title')
    tipo     = row.get('format')
    url      = row.get('url')

    
    # print(index)
    # request, resp_time = measure_response_time(cc.request_pid)
    request, resp_time, amount = measure_tx_params(dark_gw,cc.request_pid)
    try:
        ark = request.as_dict['ark']
        status = 'ok'
    except KeyError:
        ark = '-'
        status = 'error'
    
    
    oasis_dark_map[oasis_id] = {'dark' : ark, 'status' :  status , 'exec_time' : resp_time , 'amount' : amount}


    if i > 2:
        break
    i+=1

oasis_dark_map

{'UFPB-10_dac161c193eb2eddda844d329397b8d0': {'dark': '8033/fkwff300000000005h9',
  'status': 'ok',
  'exec_time': 2.0231447219848633,
  'amount': 0},
 'UFPB-10_6c85577aae298ee530508b33c13c6c25': {'dark': '8033/fkwff300000000005j3',
  'status': 'ok',
  'exec_time': 1.9804716110229492,
  'amount': 0},
 'UFPB-10_52b6d1fc4f082ea490681ce15a400d1e': {'dark': '8033/fkwff300000000005kw',
  'status': 'ok',
  'exec_time': 1.9988832473754883,
  'amount': 0},
 'UFPB-10_acb8e546ea82c55732a1ecd5adab01fb': {'dark': '8033/fkwff300000000005mp',
  'status': 'ok',
  'exec_time': 1.9857394695281982,
  'amount': 0}}

In [ ]:
# cr_client = CrossrefClient()
# cr_client.get_doi_by_title(titulo_publicacao)